In [1]:
import pyproj
pyproj.datadir.set_data_dir("/home/yifan/anaconda3/envs/myenv/share/proj")
print(pyproj.datadir.get_data_dir())

/home/yifan/anaconda3/envs/myenv/share/proj


/home/yifan/anaconda3/envs/myenv/lib/python3.9/site-packages/pyproj/__init__.py:95: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from shapely.ops import unary_union

In [3]:
# 第一步：读取2019和2020年的forest_mask文件
forest_2019 = gpd.read_file('Zone_Dataset_Mask_2020.shp')
forest_2020 = gpd.read_file('Zone_Dataset_Mask_2021.shp')

# 确保两年的数据使用相同的坐标参考系 (CRS)
if forest_2019.crs != forest_2020.crs:
    forest_2020 = forest_2020.to_crs(forest_2019.crs)

# 投影到合适的投影坐标系（例如UTM），以确保面积计算准确
projected_crs = 'EPSG:4326'  # 请根据实际情况选择合适的UTM区域
forest_2019 = forest_2019.to_crs(projected_crs)
forest_2020 = forest_2020.to_crs(projected_crs)

# 计算两年之间森林减少的区域（前一年值为1，后一年值为0）
forest_loss = gpd.overlay(forest_2019, forest_2020, how='difference')


In [4]:
# 第二步：读取手动标注的森林砍伐区域文件，并叠加
manual_deforestation_list = []
manual_deforestation_files = ['622_975_2021.4.shp', '622_975_2021.5.shp', '622_975_2021.6.shp', 
                              '622_975_2021.7.shp', '622_975_2021.8.shp', '622_975_2021.9.shp']

# 读取并验证所有手动标注的文件
for file in manual_deforestation_files:
    df = gpd.read_file(file)
    
    # 检查并修复无效几何体
    df['validity'] = df.is_valid
    if not df['validity'].all():
        print(f"Invalid geometries found in {file}, attempting to fix...")
        df['geometry'] = df['geometry'].buffer(0)  # 尝试修复无效几何体
    
    # 只保留有效的几何体
    df_valid = df[df.is_valid]
    
    if not df_valid.empty:
        manual_deforestation_list.append(df_valid)

# 检查是否有任何有效的几何体
if manual_deforestation_list:
    # 合并所有有效的手动标注的砍伐区域
    manual_deforestation = gpd.GeoDataFrame(pd.concat(manual_deforestation_list, ignore_index=True))
    
    # 如果几何体全部有效，则执行 dissolve 操作
    manual_deforestation = manual_deforestation.dissolve()
    print("Dissolve operation completed.")
else:
    print("No valid geometries were found across the shapefiles.")

Invalid geometries found in 622_975_2021.4.shp, attempting to fix...
Invalid geometries found in 622_975_2021.5.shp, attempting to fix...
Invalid geometries found in 622_975_2021.6.shp, attempting to fix...
Invalid geometries found in 622_975_2021.7.shp, attempting to fix...
Invalid geometries found in 622_975_2021.8.shp, attempting to fix...
Invalid geometries found in 622_975_2021.9.shp, attempting to fix...
Dissolve operation completed.


/tmp/ipykernel_52372/1315934556.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  manual_deforestation = gpd.GeoDataFrame(pd.concat(manual_deforestation_list, ignore_index=True))


In [ ]:
# 第三步：比较“第一步相减的结果”和“第二步相加的结果”，计算指标
# 计算真正例（TP）：两者的交集
TP = gpd.overlay(forest_loss, manual_deforestation, how='intersection')

# 计算假正例（FP）：forest_loss中减去手动标注的部分
FP = gpd.overlay(forest_loss, manual_deforestation, how='difference')

# 计算假负例（FN）：手动标注中减去forest_loss的部分
FN = gpd.overlay(manual_deforestation, forest_loss, how='difference')

# 计算面积
TP_area = TP.geometry.area.sum()
FP_area = FP.geometry.area.sum()
FN_area = FN.geometry.area.sum()

In [ ]:
# 计算评估指标
if (TP_area + FP_area) == 0:
    precision = 0
else:
    precision = TP_area / (TP_area + FP_area)

if (TP_area + FN_area) == 0:
    recall = 0
else:
    recall = TP_area / (TP_area + FN_area)

if (precision + recall) == 0:
    f1_score = 0
else:
    f1_score = 2 * (precision * recall) / (precision + recall)

if (TP_area + FP_area + FN_area) == 0:
    jaccard_index = 0
else:
    jaccard_index = TP_area / (TP_area + FP_area + FN_area)

# 输出结果
print(f'Precision（精确率）：{precision}')
print(f'Recall（召回率）：{recall}')
print(f'F1-score：{f1_score}')
print(f'Jaccard Index（杰卡德指数）：{jaccard_index}')